In [ ]:
# default_exp data

# data

> Let's do something

In [ ]:
#hide
from nbdev.showdoc import *
import argostranslate

In [ ]:
#export
default_translator_path = 'translate-en_de-1_0.argosmodel'
def generate_translator(path='translate-en_de-1_0.argosmodel'):
    '''
        path[str]: path to file from https://www.argosopentech.com/argospm/index/
    '''
    package.install_from_path('translate-en_de-1_0.argosmodel')
    installed_languages = translate.get_installed_languages()
    [str(lang) for lang in installed_languages]
    translation_en_de = installed_languages[0].get_translation(installed_languages[1])
    return translation_en_de